Définition des fonction nécessaires dans la suite

In [ ]:
import tensorflow as tf
import collections
import random as r  
import numpy as np
import codecs


def read_data(filename):
    with open(filename,"r") as f:
        data = str(f.read()).split()
        return data

def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = r.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
      # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

def save_embeddings(emb, dic, filename):
    f = open(filename, 'w')
    for i in range(len(emb)):
        f.write(dic[i])
        for j in emb[i]:
            f.write(" "+str(j))
        f.write("\n")
            
def load_embeddings(filename):
    dic = {}
    f = open(filename, "r")
    eof = False
    while not eof:
        line = f.readline()
        if line != "":
            sline = line.split()
            vec = np.float64(sline[1:])
            dic[sline[0]] = vec
        else :
            eof = True
    return dic



Partie chargement des données contenues dans le livre

In [ ]:
data = read_data("LOTR.txt")

In [ ]:
# suppression de caractères spéciaux et enregistrement du texte traité dans le fichier data.txt

file = codecs.open("data/data.txt","w")
for i in range(len(data)):
    for a in ["'","*","=","\xc2\xab","\xc2\xbb","/","\xe2\x80\x9d","\xe2\x80\x9c","\xe2\x80\x9d","\xe2\x80\x99","_","\"","\xe2\x80\x98",".",";",",","!","?",":","(",")","`"]:
        data[i] = data[i].replace(a,"")
    for a in ["\xc3\xba","\xc3\xb9","\xc3\xbb","\xc3\xbc"]:
        data[i] = data[i].replace(a,"u")
    for a in ["\xc3\xa8","\xc3\xa9","\xc3\xaa","\xc3\xab","É"]:
        data[i] = data[i].replace(a,"e")
    for a in ["\xc3\xac","\xc3\xad","\xc3\xae","\xc3\xaf"]:
        data[i] = data[i].replace(a,"i")
    for a in ["\xc3\xb2","\xc3\xb3","\xc3\xb4","\xc3\xb5","\xc3\xb6","Ó"]:
        data[i] = data[i].replace(a,"o")
    for a in ["\xc3\xa0","\xc3\xa1","\xc3\xa2","\xc3\xa3","\xc3\xa4","\xc3\xa5","À"]:
        data[i] = data[i].replace(a,"a")
    data[i] = data[i].lower()    
    file.write(data[i]+" ")
    
del data # Optimisation mémoire
        

In [ ]:
# Lecture du contenue de data.txt

vocabulary = read_data("data/data.txt")

# Etape 2: On génére nos données en s'appuyant sur le texte


vocabulary_size = 14300
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)

del vocabulary  # Fais de la place en mémoire
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])
# variable global pour savoir à quel endroit l'échantillonnage s'est arreté
data_index = 0

Partie création du modèle et entrainement

In [ ]:
def trainEmbeddings(emb_size,window, fast, steps):
    
    global vocabulary_size
    
    
    batch_size = 64
    vocab_size = vocabulary_size
    embedding_size = emb_size
    num_sampled = batch_size/2

    valid_size = 16     # Mots de référence pour vérifier l'avancement de l'entrainement
    valid_window = 100  # On choisi les mots de référence en fonction de
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)


# Données d'entrées

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  
    
    with tf.device('/cpu:0'):
    
        # Look up embeddings for inputs.
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Crée les varaibles pour laperte NCE
        nce_weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size],stddev=1.0 / np.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocab_size]))
    
      # calcul la perte NCE
 
        loss = tf.reduce_mean( tf.nn.nce_loss(weights=nce_weights,biases=nce_biases,labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocab_size))

  # Crée l'optimiseur
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    
        init = tf.global_variables_initializer()
        
    usable_embeddings = np.zeros((vocab_size,embedding_size))

    num_steps = steps
    with tf.Session() as sess:
        init.run()
        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = generate_batch(batch_size=batch_size, num_skips=window*2, skip_window=window)
            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 10000 == 0:
                if step > 0:
                    average_loss /= 10000
          # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0
                if step % 10000 == 0 and not(fast):
                    sim = similarity.eval()
                    for i in xrange(valid_size):
                        valid_word = reverse_dictionary[valid_examples[i]]
                        top_k = 3  # number of nearest neighbors
                        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                        log_str = 'Nearest to %s:' % valid_word
                        for k in xrange(top_k):
                            close_word = reverse_dictionary[nearest[k]]
                            log_str = '%s %s,' % (log_str, close_word)
                        print(log_str)
        usable_embeddings = normalized_embeddings.eval()
        return usable_embeddings

Partie sauvegarde des embeddings dans un fichier

In [ ]:
for i in range(1,4):
    for j in range(10):
        print("Window size : " + str(i) +"\nEmbeddings dim : "+str(50+10*j))
        emb = trainEmbeddings(50+10*j,i,True,10001)
        save_embeddings(emb, reverse_dictionary, "data/embW"+str(i)+"D"+str(50+10*j))